In [1]:
import os
import logging
from sklearn.model_selection import train_test_split
# Suppress warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
logging.getLogger('tensorflow').setLevel(logging.ERROR)

# set backend
os.environ['KERAS_BACKEND'] = 'torch'
import keras
print("Backend after setting:", keras.config.backend())

import tensorflow as tf
import torch

# Check GPU visibility
# print("TensorFlow GPUs:", tf.config.list_physical_devices('GPU'))
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

Backend after setting: torch


In [2]:
class Config:
    def __init__(self, device_id=0):
        self.device = torch.device(f"cuda:{device_id}" if torch.cuda.is_available() else "cpu")

# Set device to GPU 1
config = Config(device_id=1)
device = config.device
torch.cuda.set_device(config.device.index)
print(torch.cuda.current_device()) 
print("PyTorch Device:", device)


1
PyTorch Device: cuda:1


In [3]:
# from keras.models import Sequential
# from keras import regularizers
# from keras.layers import (Input, Conv2D, BatchNormalization, ReLU, MaxPooling2D, 
#                           Flatten, Dense, Dropout, Lambda)
# from keras.initializers import HeNormal
# import keras.ops as K

# def get_model(hidden_units, output_units, input_shape, rate, l2_coeff=1e-5):
#     """
#     Creates a face verification model that outputs normalized embeddings.
#     """

#     model = Sequential([Input(shape=input_shape)])

#     # --- Convolutional blocks / Feature extraction backbone ---

#     # note we use he kaiming initialization for the weights
#     model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer=HeNormal(),
#                      kernel_regularizer=regularizers.l2(l2_coeff)))
#     model.add(BatchNormalization())
#     model.add(ReLU())
#     model.add(MaxPooling2D((2, 2)))

#     # 2nd block
#     model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer=HeNormal(),
#                      kernel_regularizer=regularizers.l2(l2_coeff)))
#     model.add(BatchNormalization())
#     model.add(ReLU())
#     model.add(MaxPooling2D((2, 2)))

    # # 3rd block
    # model.add(Conv2D(128, (3, 3), padding='same', kernel_initializer=HeNormal(),
    #                  kernel_regularizer=regularizers.l2(l2_coeff)))
    # model.add(BatchNormalization())
    # model.add(ReLU())
    # model.add(MaxPooling2D((2, 2)))

    # model.add(Flatten())

    # # --- Fully connected layers ---
    # for units in hidden_units:
    #     model.add(Dense(units, kernel_initializer=HeNormal(),
    #                     kernel_regularizer=regularizers.l2(l2_coeff)))
    #     model.add(BatchNormalization())
    #     model.add(ReLU())
    #     model.add(Dropout(rate))

    # # --- Output layer + normalization ---
    # model.add(Dense(output_units, kernel_initializer=HeNormal()))
    # model.add(Lambda(lambda x: x / keras.ops.norm(x, axis=1, keepdims=True)))

    # return model

# model = get_model(
#     hidden_units=[1024, 128],
#     output_units=128,
#     input_shape=(112, 112, 3),
#     rate=0.5
# )
# # some random input
# x = torch.randn(1, 112, 112, 3).to(device)
# model(x).shape

In [4]:
# from keras.models import Sequential
# from keras import regularizers
# from keras.layers import (Input, Conv2D, BatchNormalization, ReLU, MaxPooling2D, 
#                           AveragePooling2D, Flatten, Dense, Dropout, Lambda, concatenate)
# from keras.initializers import HeNormal
# import keras.ops as K
# import keras

# def inception_module(x, filters, l2_coeff=1e-5):
#     """
#     Creates an Inception module with parallel conv layers.
    
#     Args:
#         x (keras Layer): Input layer.
#         filters (tuple): Number of filters for (1x1, 3x3 reduce, 3x3, 5x5 reduce, 5x5, pool_proj).
#         l2_coeff (float): L2 regularization coefficient.
    
#     Returns:
#         keras Layer: Output of Inception module.
#     """
#     f1, f3r, f3, f5r, f5, proj = filters

#     # 1x1 Convolution
#     conv1x1 = Conv2D(f1, (1,1), padding='same', kernel_initializer=HeNormal(),
#                      kernel_regularizer=regularizers.l2(l2_coeff))(x)
#     conv1x1 = BatchNormalization()(conv1x1)
#     conv1x1 = ReLU()(conv1x1)

#     # 3x3 Convolution
#     conv3x3 = Conv2D(f3r, (1,1), padding='same', kernel_initializer=HeNormal(),
#                      kernel_regularizer=regularizers.l2(l2_coeff))(x)
#     conv3x3 = BatchNormalization()(conv3x3)
#     conv3x3 = ReLU()(conv3x3)
#     conv3x3 = Conv2D(f3, (3,3), padding='same', kernel_initializer=HeNormal(),
#                      kernel_regularizer=regularizers.l2(l2_coeff))(conv3x3)
#     conv3x3 = BatchNormalization()(conv3x3)
#     conv3x3 = ReLU()(conv3x3)

#     # 5x5 Convolution
#     conv5x5 = Conv2D(f5r, (1,1), padding='same', kernel_initializer=HeNormal(),
#                      kernel_regularizer=regularizers.l2(l2_coeff))(x)
#     conv5x5 = BatchNormalization()(conv5x5)
#     conv5x5 = ReLU()(conv5x5)
#     conv5x5 = Conv2D(f5, (5,5), padding='same', kernel_initializer=HeNormal(),
#                      kernel_regularizer=regularizers.l2(l2_coeff))(conv5x5)
#     conv5x5 = BatchNormalization()(conv5x5)
#     conv5x5 = ReLU()(conv5x5)

#     # L2 Pooling followed by 1x1 conv
#     pool = AveragePooling2D((3,3), strides=(1,1), padding='same')(x)  # L2 pooling
#     pool = Conv2D(proj, (1,1), padding='same', kernel_initializer=HeNormal(),
#                   kernel_regularizer=regularizers.l2(l2_coeff))(pool)
#     pool = BatchNormalization()(pool)
#     pool = ReLU()(pool)

#     # Concatenate all branches
#     output = concatenate([conv1x1, conv3x3, conv5x5, pool])

#     return output


# def get_inception_nn2(input_shape=(112, 112, 3), num_classes=128, l2_coeff=1e-5, dropout_rate=0.5):
#     """
#     Builds the Inception-based NN2 model using Keras Sequential API.
#     """

#     inputs = Input(shape=input_shape)
#     x = Conv2D(64, (7,7), strides=2, padding='same', kernel_initializer=HeNormal(),
#                kernel_regularizer=regularizers.l2(l2_coeff))(inputs)
#     x = BatchNormalization()(x)
#     x = ReLU()(x)
#     x = MaxPooling2D((3,3), strides=2, padding='same')(x)

#     # Inception modules
#     x = inception_module(x, (64, 96, 128, 16, 32, 32))  # Inception 3a
#     x = inception_module(x, (64, 96, 128, 32, 64, 64))  # Inception 3b
#     x = MaxPooling2D((3,3), strides=2, padding='same')(x)

#     x = inception_module(x, (128, 128, 256, 32, 64, 64))  # Inception 4a
#     x = inception_module(x, (128, 128, 256, 48, 128, 64))  # Inception 4b
#     x = inception_module(x, (128, 144, 288, 48, 128, 64))  # Inception 4c
#     x = MaxPooling2D((3,3), strides=2, padding='same')(x)

#     x = inception_module(x, (256, 160, 320, 48, 128, 128))  # Inception 5a
#     x = inception_module(x, (256, 160, 320, 48, 128, 128))  # Inception 5b
#     x = AveragePooling2D((7,7))(x)

#     # Flatten and Fully Connected Layers
#     x = Flatten()(x)
#     x = Dense(1024, kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(l2_coeff))(x)
#     x = BatchNormalization()(x)
#     x = ReLU()(x)
#     x = Dropout(dropout_rate)(x)

#     x = Dense(num_classes, kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(l2_coeff))(x)
    
#     # Normalize the output embeddings
#     outputs = Lambda(lambda x: x / K.norm(x, axis=1, keepdims=True))(x)

#     model = keras.Model(inputs, outputs, name="Inception_NN2")

#     return model


# # Create model
# model = get_inception_nn2()
# model.summary()

In [5]:
from keras.models import Model, Sequential
from keras import regularizers
from keras.layers import (Input, Conv2D, BatchNormalization, ReLU, MaxPooling2D, 
                          AveragePooling2D, Flatten, Dense, Dropout, Lambda, concatenate)
from keras.initializers import HeNormal
import keras.ops as K
import keras

def inception_module(x, filters, l2_coeff=1e-5):
    """
    Creates an Inception module with parallel conv layers.
    """
    f1, f3r, f3, f5r, f5, proj = filters

    # 1x1 Convolution
    conv1x1 = Conv2D(f1, (1,1), padding='same', kernel_initializer=HeNormal(),
                     kernel_regularizer=regularizers.l2(l2_coeff))(x)
    conv1x1 = BatchNormalization()(conv1x1)
    conv1x1 = ReLU()(conv1x1)

    # 3x3 Convolution
    conv3x3 = Conv2D(f3r, (1,1), padding='same', kernel_initializer=HeNormal(),
                     kernel_regularizer=regularizers.l2(l2_coeff))(x)
    conv3x3 = BatchNormalization()(conv3x3)
    conv3x3 = ReLU()(conv3x3)
    conv3x3 = Conv2D(f3, (3,3), padding='same', kernel_initializer=HeNormal(),
                     kernel_regularizer=regularizers.l2(l2_coeff))(conv3x3)
    conv3x3 = BatchNormalization()(conv3x3)
    conv3x3 = ReLU()(conv3x3)

    # 5x5 Convolution
    conv5x5 = Conv2D(f5r, (1,1), padding='same', kernel_initializer=HeNormal(),
                     kernel_regularizer=regularizers.l2(l2_coeff))(x)
    conv5x5 = BatchNormalization()(conv5x5)
    conv5x5 = ReLU()(conv5x5)
    conv5x5 = Conv2D(f5, (5,5), padding='same', kernel_initializer=HeNormal(),
                     kernel_regularizer=regularizers.l2(l2_coeff))(conv5x5)
    conv5x5 = BatchNormalization()(conv5x5)
    conv5x5 = ReLU()(conv5x5)

    # Pooling followed by 1x1 conv
    pool = AveragePooling2D((3,3), strides=(1,1), padding='same')(x)
    pool = Conv2D(proj, (1,1), padding='same', kernel_initializer=HeNormal(),
                  kernel_regularizer=regularizers.l2(l2_coeff))(pool)
    pool = BatchNormalization()(pool)
    pool = ReLU()(pool)

    # Concatenate all branches
    return concatenate([conv1x1, conv3x3, conv5x5, pool])


def get_inception_nn2(input_shape=(112, 112, 3), num_classes=128, l2_coeff=1e-5, dropout_rate=0.5):
    """
    Builds the Inception-based NN2 model using Keras Model API.
    """

    inputs = Input(shape=input_shape)

    # Sequential-style additions
    x = Conv2D(64, (7,7), strides=2, padding='same', kernel_initializer=HeNormal(),
               kernel_regularizer=regularizers.l2(l2_coeff))(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((3,3), strides=2, padding='same')(x)

    # Inception modules
    x = inception_module(x, (64, 96, 128, 16, 32, 32))  # Inception 3a
    x = inception_module(x, (64, 96, 128, 32, 64, 64))  # Inception 3b
    x = MaxPooling2D((3,3), strides=2, padding='same')(x)

    x = inception_module(x, (128, 128, 256, 32, 64, 64))  # Inception 4a
    x = inception_module(x, (128, 128, 256, 48, 128, 64))  # Inception 4b
    x = inception_module(x, (128, 144, 288, 48, 128, 64))  # Inception 4c
    x = MaxPooling2D((3,3), strides=2, padding='same')(x)

    x = inception_module(x, (256, 160, 320, 48, 128, 128))  # Inception 5a
    x = inception_module(x, (256, 160, 320, 48, 128, 128))  # Inception 5b
    x = AveragePooling2D((7,7))(x)

    # Flatten and Fully Connected Layers
    x = Flatten()(x)
    x = Dense(1024, kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(l2_coeff))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Dropout(dropout_rate)(x)

    x = Dense(num_classes, kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(l2_coeff))(x)
    
    # Normalize the output embeddings
    outputs = Lambda(lambda x: x / K.norm(x, axis=1, keepdims=True))(x)

    model = Model(inputs, outputs, name="Inception_NN2")

    return model


# Create model
model = get_inception_nn2()
model.summary()


Model: "Inception_NN2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 112, 112,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 56, 56,    │      9,472 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 56, 56,    │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 28, 28,    │          0 │ re_lu[0][0]       │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 28, 28,    │      6,240 │ max_pooling2d[0]… │
│                     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 28, 28,    │      1,040 │ max_pooling2d[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 28, 28,    │        384 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 28, 28,    │         64 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 28, 28,    │          0 │ batch_normalizat… │
│                     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_4 (ReLU)      │ (None, 28, 28,    │          0 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_pooling2d   │ (None, 28, 28,    │          0 │ max_pooling2d[0]… │
│ (AveragePooling2D)  │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 28, 28,    │      4,160 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 28, 28,    │    110,720 │ re_lu_2[0][0]     │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 28, 28,    │     12,832 │ re_lu_4[0][0]     │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 28, 28,    │      2,080 │ average_pooling2… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 28, 28,    │        256 │ conv2d_1[0][0]  

 Total params: 5,312,640 (20.27 MB)

 Trainable params: 5,300,224 (20.22 MB)

 Non-trainable params: 12,416 (48.50 KB)

In [6]:
# some random input
x = torch.randn(1, 112, 112, 3).to(device)
model(x).shape

torch.Size([1, 128])

total_data_size = ideneitities * max_per_identity = 8000 * 10

each batch need 10 faces per identity, so each batch need 8000 pics

In [7]:
import os
import random
import numpy as np
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader, Sampler

class PytorchFaceDataset(Dataset):
    """
    A single unified class that:
      - Loads up to `max_images_per_identity` images from each identity folder.
      - Optionally uses `sample_indexes` for train/val slicing.
    """
    def __init__(
        self, dataset_path, identities,
        max_images_per_identity=10,
        sample_indexes=None,
        classes_per_batch=None,
        samples_per_class=None
    ):
        self.dataset_path = dataset_path
        self.identities = identities
        self.max_images_per_identity = max_images_per_identity
        self.sample_indexes = sample_indexes
        self.classes_per_batch = classes_per_batch
        self.samples_per_class = samples_per_class

        self.image_paths = []
        self.labels = []  # We'll group by label = idx in `identities`
        
        # Gather all images & labels
        for idx, identity in enumerate(identities):
            identity_folder = os.path.join(dataset_path, identity)
            if not os.path.isdir(identity_folder):
                continue  # skip if folder doesn't exist

            image_files = sorted(os.listdir(identity_folder))
            # Take up to max_images_per_identity images
            selected_images = image_files[:max_images_per_identity]
            for img_name in selected_images:
                self.image_paths.append(os.path.join(identity_folder, img_name))
                self.labels.append(idx)

        # sample_indexes is train_indices or val_indices
        if self.sample_indexes is not None:
            self.image_paths = [self.image_paths[i] for i in self.sample_indexes]
            self.labels = [self.labels[i] for i in self.sample_indexes]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        img_path = self.image_paths[index]
        label = self.labels[index]

        # Load image, convert to RGB, resize to (112, 112)
        img = Image.open(img_path).convert("RGB")
        img = img.resize((112, 112), resample=Image.BILINEAR)
        # Normalize to [0, 1]
        img = np.array(img, dtype=np.float32) / 255.0  # shape: (112, 112, 3)

        img_tensor = torch.from_numpy(img)  # currently (112, 112, 3)

        return img_tensor, label

    def get_dataloader(self, batch_size=128, shuffle=True, num_workers=4):
        # If we have custom-batch parameters set, build the custom sampler
        if self.classes_per_batch is not None and self.samples_per_class is not None:
            return DataLoader(
                self,
                batch_sampler=self._BatchSampler(
                    self.labels,
                    self.classes_per_batch,
                    self.samples_per_class
                ),
                num_workers=num_workers
            )
        else:
            return DataLoader(
                self,
                batch_size=batch_size,
                shuffle=shuffle,
                num_workers=num_workers
            )

    class _BatchSampler(Sampler):
        """
        Ensures each batch contains `classes_per_batch` classes, each has
        `samples_per_class` samples or fewer.
        """
        def __init__(self, labels, classes_per_batch, samples_per_class):
            self.labels = labels
            self.classes_per_batch = classes_per_batch
            self.samples_per_class = samples_per_class

            # Group indices by class
            self.class_to_indices = {}
            for idx, label in enumerate(labels):
                self.class_to_indices.setdefault(label, []).append(idx)

            # Keep list of all classes for shuffling each epoch
            self.all_classes = list(self.class_to_indices.keys())

        def __iter__(self):
            '''Defines how the batch indices are generated.'''
            random.shuffle(self.all_classes) # Shuffle list of all labels each epoch

            # We'll chunk the shuffled class list in groups of 'classes_per_batch'

            # loop over shuffled classes in chunks of 'classes_per_batch'
            # e.g. 8000 classes, 10 class per epoch, so 800 iterations
            for start in range(0, len(self.all_classes), self.classes_per_batch):
                chunk_classes = self.all_classes[start:start + self.classes_per_batch]
                
                batch_indices = []
                for cls in chunk_classes: # collect sample indices for each class
                    idx_list = self.class_to_indices[cls] # map label to indices

                    # if a class has enough samples, sample 'samples_per_class' indices
                    # otherwise, take all indices
                    if len(idx_list) >= self.samples_per_class:
                        chosen = random.sample(idx_list, self.samples_per_class)
                    else:
                        chosen = idx_list  # class is smaller than desired
                    batch_indices.extend(chosen)

                yield batch_indices # move to dataloader

        def __len__(self):
            ''' computes how many batches are needed to cover all classes '''
            # e.g. 8000 class, 10 class per batch, so (8010-1) // 10 = 800
            return (len(self.all_classes) + self.classes_per_batch - 1) // self.classes_per_batch


Note we have 8000 identities, and choose 10 images from each ideneity, so total size = 80k

We have 800 batches, and within each batch, 10 classes and $\leq$ 10 samples from each class

In [8]:
# # Load dataset identities
dataset_path = "data/casia-webface"
identities = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]
print("Number of identities in dataset:", len(identities))
indices = np.arange(80000)
train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42)
train_torch_dataset = PytorchFaceDataset(dataset_path, identities, 
                                         max_images_per_identity=10, 
                                         sample_indexes=train_indices,
                                         classes_per_batch=10,   # each batch will pick 10 classes
                                        samples_per_class=10) # each class has 10 samples
train_loader = train_torch_dataset.get_dataloader(num_workers=4) # batch size = 10*10=100

val_torch_dataset = PytorchFaceDataset(dataset_path, identities,
                                       max_images_per_identity=10, 
                                       sample_indexes=val_indices,
                                       classes_per_batch=10,   
                                       samples_per_class=10) 

val_loader = val_torch_dataset.get_dataloader(num_workers=4) # batch size = 10*10=100

# Check shape
for images, labels in train_loader:
    print(f"Images shape: {images.shape}, Labels shape: {labels.shape}")
    print("Unique labels in this batch:", len(set(labels.tolist())))
    break


Number of identities in dataset: 8000


/usr/lib64/python3.9/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib64/python3.9/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Images shape: torch.Size([84, 112, 112, 3]), Labels shape: torch.Size([84])
Unique labels in this batch: 10


In [9]:
import torch
import torch.nn.functional as F

def circle_loss_torch(anchor_embeddings, positive_embeddings, negative_embeddings, m=0.25, gamma=64.0):
    # -- Step 1: Compute scaled cosine similarities in [0, 1].
    s_p = ((anchor_embeddings * positive_embeddings).sum(dim=1) + 1.0) / 2.0
    s_n = ((anchor_embeddings * negative_embeddings).sum(dim=1) + 1.0) / 2.0

    alpha_p = 1.0 + m - s_p
    alpha_n = s_n + m
    delta_p = 1.0 - m
    delta_n = m

    # We'll compute this in a vectorized way and then average across the batch.
    inside_term = gamma * (alpha_n * (s_n - delta_n) - alpha_p * (s_p - delta_p))
    losses = (1.0 / gamma) * torch.log1p(torch.exp(inside_term))  # log1p(x) = log(1 + x)
    
    return losses.mean()


The idea is to dynamically generate triplets during training

In [10]:
def get_triplets(model, batch_data, margin=0.2, validation=False):
    """Selects hard triplets from a batch for triplet loss."""
    images, labels = batch_data  # Unpack batch
    labels = labels.cpu().numpy()  # Convert labels to numpy array

    with torch.no_grad():
        embeddings = model(images)
    
    # Compute pairwise distances
    batch_size = embeddings.shape[0] # should be 1024
    distances = torch.cdist(embeddings, embeddings, p=2)  # Euclidean dist

    triplets = []
    
    for anchor_idx in range(batch_size):
        anchor_label = labels[anchor_idx]

        # Find positive indices
        positive_indices = np.where(labels == anchor_label)[0]
        positive_indices = positive_indices[positive_indices != anchor_idx]  # Exclude anchor

        # print(len(positive_indices), 'positive indices length')

        # Find negative indices
        negative_indices = np.where(labels != anchor_label)[0]
        
        # print(len(negative_indices), 'negative indices length')

        if len(positive_indices) == 0 or len(negative_indices) == 0:
            continue  # Skip if we can't form a triplet

        # Select the hardest positive (farthest within same class)
        hardest_positive_idx = max(positive_indices, key=lambda i: distances[anchor_idx, i].item())

        # semi-hard negative, need to have bigger distance than hardest positive
        semi_hard_negatives = [i for i in negative_indices if distances[anchor_idx, i] > distances[anchor_idx, hardest_positive_idx]]
        if len(semi_hard_negatives) == 0:
            continue
        else:
            # neg_inx = random.choice(semi_hard_negatives) # randomly choose one negative
            hardest_negative_idx = min(semi_hard_negatives, key=lambda i: distances[anchor_idx, i].item())

        # print("Positive distance:", distances[anchor_idx, hardest_positive_idx])
        # print("hardest Negative distance:", distances[anchor_idx, hardest_negative_idx])

        if validation:
            # Don't need margin condition
            neg_inx = random.choice(negative_indices)
            triplets.append((anchor_idx, hardest_positive_idx, neg_inx))
            return triplets
        
        # For training, need to ensure valid triplet (margin condition)
        if distances[anchor_idx, hardest_negative_idx] - distances[anchor_idx, hardest_positive_idx] > margin:
            triplets.append((anchor_idx, hardest_positive_idx, hardest_negative_idx))

    return triplets


In [11]:
for batch, labels in train_loader:
    print(batch.shape, 'batch shape')
    print(labels.shape, 'labels shape')
    break

torch.Size([73, 112, 112, 3]) batch shape
torch.Size([73]) labels shape


In [12]:
for images, labels in train_loader:
    triplets = get_triplets(model, (images.to(device), labels.to(device)), margin=1e-5)
    print(f"Number of triplets: {len(triplets)}")
    print("Sample triplet:", triplets[0])
    anchor = triplets[0][0]
    print(images[anchor].shape) # anchor image shape
    break

Number of triplets: 72
Sample triplet: (0, 8, 21)
torch.Size([112, 112, 3])


In [13]:
def pt_train_step(mlp_model, loss_fn, opt, train_batch):
    """PyTorch training step for triplet loss."""
    mlp_model.zero_grad()

    (anchor, positive, negative) = train_batch

    anchor_embed = mlp_model(anchor)  # Shape: (batch_size, embedding_dim)
    positive_embed = mlp_model(positive)
    negative_embed = mlp_model(negative)

    loss = loss_fn(anchor_embed, positive_embed, negative_embed)

    loss.backward()

    # grads = [param.grad for param in mlp_model.parameters()]
    grads = [v.value.grad for v in mlp_model.trainable_variables]

    return loss, grads


In [14]:
def pt_valid_step(mlp_model, val_batch):
    """PyTorch validation step for triplet loss."""
    (anchor, positive, negative) = val_batch

    anchor_embed = mlp_model(anchor)  # Shape: (batch_size, embedding_dim)
    positive_embed = mlp_model(positive)
    negative_embed = mlp_model(negative)
    
    pos_dist = torch.norm(anchor_embed - positive_embed, dim=1)  # Distance A→P
    neg_dist = torch.norm(anchor_embed - negative_embed, dim=1)  # Distance A→N
    # check what percentage of data has anchor_embed closer to positive_embed than negative_embed


    correct = (pos_dist < neg_dist).float()  # 1 if correct, 0 if incorrect
    accuracy = correct.mean().item()
    return accuracy

In [15]:
import keras
loss_metric = keras.metrics.Mean()
accuracy_metric = keras.metrics.SparseCategoricalAccuracy()
val_accuracy_metric = keras.metrics.Mean() 
# val_loss_metric = keras.metrics.Mean()

In [16]:
def train_model_custom(mlp_model, loss_fn, opt, training_dataset, validation_dataset, train_step_fn, valid_step_fn, epochs):
    """
    Train the model and evaluate on a validation set.
    Returns lists of training and validation losses/accuracies.
    """
    # check if training is using GPU
    # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    device = config.device
    print(f"Device: {device}")
    mlp_model.to(device)
    print("Training on GPU:", next(mlp_model.parameters()).is_cuda)

    epoch_losses = []
    epoch_acc = []
    val_epoch_losses = []
    val_epoch_acc = []

    for epoch in range(epochs):
        print(f"Epoch {epoch}/{epochs}")
        # Reset metrics for new epoch
        loss_metric.reset_state()
        val_accuracy_metric.reset_state()
        
        # Training loop
        for images, labels in training_dataset:
            triplets = get_triplets(model, (images.to(device), labels.to(device)), margin=1e-5) # triplet indices
            anchors = [triplet[0] for triplet in triplets] # anchor indices
            positives = [triplet[1] for triplet in triplets] # positive indices
            negatives = [triplet[2] for triplet in triplets] # negative indices

            triplet_data = (images[anchors], images[positives], images[negatives])

            loss, grads = train_step_fn(mlp_model, loss_fn, opt, train_batch=triplet_data)



            # grads_vars = [(grad, var) for grad, var in zip(grads, mlp_model.trainable_variables)]

            # for grad, var in zip(grads, mlp_model.trainable_variables):
            #     if grad is not None:
            #         print(f"Grad shape: {grad.shape}, Var shape: {var.shape}")

            opt.apply_gradients(zip(grads, mlp_model.trainable_variables))
                        
            # opt.apply_gradients(grads_vars)

            
            loss_metric.update_state(loss)

        # Compute training loss and accuracy
        avg_epoch_loss = float(loss_metric.result().cpu().numpy())

        # Validation loop
        with torch.no_grad():  # Disable gradients for validation
            for images, labels in validation_dataset:
                triplets = get_triplets(model, (images.to(device), labels.to(device)), margin=1e-5, validation=True)
                anchors = [triplet[0] for triplet in triplets]
                positives = [triplet[1] for triplet in triplets]
                negatives = [triplet[2] for triplet in triplets]
                triplet_data = (images[anchors], images[positives], images[negatives])
                acc = valid_step_fn(mlp_model, val_batch=triplet_data)
                val_accuracy_metric.update_state(acc)

        # Compute validation loss and accuracy
        avg_val_acc = float(val_accuracy_metric.result().cpu().numpy())

        # Store epoch results
        epoch_losses.append(avg_epoch_loss)
        # val_epoch_losses.append(avg_val_loss)
        val_epoch_acc.append(avg_val_acc)

        # Print progress
        print(f"Epoch {epoch}: loss - {avg_epoch_loss:.4f}, ")

        # for param in model.parameters():
        #     print('grad:', param.grad)

        # save checkpoint
        # if epoch % 10 == 0:
        print(f"val_acc - {avg_val_acc:.4f}")

        checkpoint_dir = "checkpoint"
        os.makedirs(checkpoint_dir, exist_ok=True)

        # Save the model
        mlp_model.save(os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}.keras"))

        # Save additional metadata separately (like epoch & val accuracy)
        metadata = {
            'epoch': epoch,
            'val_accuracy': avg_val_acc
        }
        import json
        with open(os.path.join(checkpoint_dir, f"checkpoint_metadata_epoch_{epoch}.json"), "w") as f:
            json.dump(metadata, f)



    return epoch_losses


In [17]:
optimizer = keras.optimizers.SGD(learning_rate=0.05)

epoch_losses = train_model_custom(model, loss_fn=circle_loss_torch, opt=optimizer,
                                            training_dataset=train_loader, 
                                            validation_dataset=val_loader, 
                                            train_step_fn=pt_train_step, 
                                            valid_step_fn=pt_valid_step,
                                            epochs=5)


Device: cuda:1
Training on GPU: True
Epoch 0/5
Epoch 0: loss - 0.4132, 
val_acc - 0.4902
Epoch 1/5
Epoch 1: loss - 0.3812, 
val_acc - 0.4874
Epoch 2/5
Epoch 2: loss - 0.3804, 
val_acc - 0.5014
Epoch 3/5
Epoch 3: loss - 0.3798, 
val_acc - 0.5531
Epoch 4/5
Epoch 4: loss - 0.3817, 
val_acc - 0.5363


In [29]:
mlp_model = get_inception_nn2()

optimizer = torch.optim.Adam(mlp_model.parameters(), lr=1e-3)  # Define optimizer

# Create dummy input and target
dummy_input = torch.randn(1, 112, 112, 3)  # Adjust shape to match model input
dummy_target = torch.tensor([1])  # Assuming single-class classification

dummy_input, dummy_target = dummy_input.to(device), dummy_target.to(device)
# Forward pass
output = mlp_model(dummy_input)

# Compute loss (modify based on actual loss function)
loss_fn = torch.nn.CrossEntropyLoss()
loss = loss_fn(output, dummy_target)

# Backward pass
optimizer.zero_grad()  # Reset gradients before backward
loss.backward()





for name, param in mlp_model.named_parameters():
    if param.grad is not None:
        print(f"Grad shape for {name}: {param.grad.shape}, Var shape: {param.shape}")
    else:
        print(f"Grad shape for {name}: None, Var shape: {param.shape}")

Grad shape for _torch_params.batch_normalization_264/beta: torch.Size([64]), Var shape: torch.Size([64])
Grad shape for _torch_params.batch_normalization_264/gamma: torch.Size([64]), Var shape: torch.Size([64])
Grad shape for _torch_params.batch_normalization_264/moving_mean: None, Var shape: torch.Size([64])
Grad shape for _torch_params.batch_normalization_264/moving_variance: None, Var shape: torch.Size([64])
Grad shape for _torch_params.batch_normalization_265/beta: torch.Size([64]), Var shape: torch.Size([64])
Grad shape for _torch_params.batch_normalization_265/gamma: torch.Size([64]), Var shape: torch.Size([64])
Grad shape for _torch_params.batch_normalization_265/moving_mean: None, Var shape: torch.Size([64])
Grad shape for _torch_params.batch_normalization_265/moving_variance: None, Var shape: torch.Size([64])
Grad shape for _torch_params.batch_normalization_266/beta: torch.Size([96]), Var shape: torch.Size([96])
Grad shape for _torch_params.batch_normalization_266/gamma: torch

In [ ]:
import os

pairs_txt = "./data/lfw_test_pairs.txt"   
lfw_root  = "./data/labeled-faces-in-the-wild"       

pairs = []  # will store tuples of (image1_path, image2_path, label)
            # where label = 1 for same-person (positive), 0 for different-person (negative)

with open(pairs_txt, 'r') as f:
    first_line = f.readline().strip()
    n_folds, n_pairs_per_fold = first_line.split() # 10 300
    n_folds, n_pairs_per_fold = int(n_folds), int(n_pairs_per_fold)

    # For each of the 10 folds:
    for _ in range(n_folds):
        # 1) Parse the POSITIVE (same-person) pairs => 3 columns each
        for _ in range(n_pairs_per_fold):
            line = f.readline().strip().split()
            person   = line[0]
            img_idx1 = int(line[1])
            img_idx2 = int(line[2])
            label    = 1  # same person

            img1_path = os.path.join(lfw_root, person, f"{person}_{img_idx1:04d}.jpg")
            img2_path = os.path.join(lfw_root, person, f"{person}_{img_idx2:04d}.jpg")

            pairs.append((img1_path, img2_path, label))

        # 2) Parse the NEGATIVE (different-person) pairs => 4 columns each
        for _ in range(n_pairs_per_fold):
            line = f.readline().strip().split()
            person1   = line[0]
            img1_idx  = int(line[1])
            person2   = line[2]
            img2_idx  = int(line[3])
            label     = 0  # different persons

            img1_path = os.path.join(lfw_root, person1, f"{person1}_{img1_idx:04d}.jpg")
            img2_path = os.path.join(lfw_root, person2, f"{person2}_{img2_idx:04d}.jpg")

            pairs.append((img1_path, img2_path, label))

# Now `pairs` is a list of (image1_path, image2_path, label).

print(f"Total pairs parsed = {len(pairs)}")
print("Example pairs:")
for i in range(3):
    print(pairs[i])


In [16]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt

class LFWDataset(Dataset):

    def __init__(self, pairs):
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)
    
    def preprocess_image(self, img_path):
        img = Image.open(img_path).convert('RGB')  # Ensure 3-channel RGB image
        img = img.resize((112, 112), resample=Image.BILINEAR)
        # Normalize to [0, 1]
        img = np.array(img, dtype=np.float32) / 255.0  # shape: (112, 112, 3)
        img_tensor = torch.from_numpy(img)
        
        return img_tensor

    def __getitem__(self, idx):
        img1_path, img2_path, label = self.pairs[idx]

        img1 = self.preprocess_image(img1_path)
        img2 = self.preprocess_image(img2_path)

        if img1 is None or img2 is None:
            return None, None, None  # Skip if missing images

        return img1, img2, torch.tensor(label, dtype=torch.float32)
    
def get_dataloader(pairs, batch_size, num_workers=4):
    """Creates a PyTorch DataLoader for processing image pairs in parallel."""
    dataset = LFWDataset(pairs)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    return dataloader

def extract_embeddings(model, dataloader):
    # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    device = config.device
    print(f"Device: {device}")
    model.to(device)
    model.eval()

    embeddings1 = []
    embeddings2 = []
    labels = []

    with torch.no_grad():
        printed = False
        for img1, img2, label in dataloader:
            if img1 is None or img2 is None:
                print("Skipping a batch due to missing images.")
                continue

            img1 = img1.to(device)
            img2 = img2.to(device)
            if not printed:
                print('img shape:', img1.shape, img2.shape)


            emb1 = model(img1)  # Output shape: (batch_size, embedding_dim)
            emb2 = model(img2)

            if not printed:
                print('emb shape:', emb1.shape, emb2.shape)
                printed = True

            embeddings1.append(emb1)
            embeddings2.append(emb2)
            labels.extend(label.cpu().numpy())

    embeddings1 = torch.cat(embeddings1, dim=0)  # Shape: (N, embedding_dim)
    embeddings2 = torch.cat(embeddings2, dim=0)  # Shape: (N, embedding_dim)

    return embeddings1, embeddings2, np.array(labels)
def compute_similarity(embeddings1, embeddings2, labels):
    """Computes scaled cosine similarity for all image pairs and plots histograms."""

    embeddings1 = F.normalize(embeddings1, p=2, dim=1)  # Ensures ||x_1|| = 1
    embeddings2 = F.normalize(embeddings2, p=2, dim=1)  # Ensures ||x_2|| = 1

    # Compute cosine similarity for entire batch
    cosine_similarities = F.cosine_similarity(embeddings1, embeddings2, dim=1).cpu().numpy()  # Shape: (N,)
    similarities = (cosine_similarities + 1) / 2

    # Split into positive and negative pairs
    positive_scores = similarities[labels == 1]
    negative_scores = similarities[labels == 0]

    # Plot histograms
    plt.figure(figsize=(8, 6))
    bins = 30

    plt.hist(positive_scores, bins=bins, alpha=0.5, label='Same Person')
    plt.hist(negative_scores, bins=bins, alpha=0.5, label='Different Person')

    plt.title("Distribution of Cosine Similarities on LFW")
    plt.xlabel("Cosine Similarity")
    plt.ylabel("Frequency")
    plt.legend()
    plt.show()

    return similarities



In [ ]:
batch_size = 32
dataloader = get_dataloader(pairs, batch_size=batch_size, num_workers=4)
print('dataloader created!')
embed1, embed2, labels = extract_embeddings(model, dataloader)
print('embeddings extracted!')
print(len(embed1), len(labels))

In [ ]:
# Compute similarity and visualize
similarities = compute_similarity(embed1, embed2, labels)

In [ ]:
def compute_auc_metrics(similarity_scores, labels):
    auc_roc = tf.keras.metrics.AUC(curve='ROC')
    auc_pr = tf.keras.metrics.AUC(curve='PR')
    
    auc_roc.update_state(labels, similarity_scores)
    auc_pr.update_state(labels, similarity_scores)
    
    return auc_roc.result().cpu().numpy(), auc_pr.result().cpu().numpy()


roc_auc, pr_auc = compute_auc_metrics(similarities, labels)
print(f"ROC AUC: {roc_auc:.4f}")
print(f"PR AUC: {pr_auc:.4f}")


In [ ]:
from sklearn.metrics import accuracy_score

# Finding the optimal threshold
thresholds = np.linspace(0, 1, 1000)
accuracies = []

for threshold in thresholds:
    predictions = (similarities >= threshold).astype(int)  # Convert similarity to binary prediction
    acc = accuracy_score(labels, predictions)
    accuracies.append(acc)

# Get the best threshold
best_idx = np.argmax(accuracies)
best_threshold = thresholds[best_idx]
best_accuracy = accuracies[best_idx]

print(f"Optimal Threshold: {best_threshold:.4f}")
print(f"Best Accuracy: {best_accuracy:.4f}")

# Plot accuracy vs. threshold
plt.figure(figsize=(6, 5))
plt.plot(thresholds, accuracies, label='Accuracy')
plt.axvline(x=best_threshold, color='r', linestyle='--', label=f'Best Threshold = {best_threshold:.4f}')
plt.xlabel('Threshold')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Threshold')
plt.legend()
plt.grid()
plt.show()
